# A DSL alongside a Genetic Algorithm applied to the ARC Dataset

In this notebook, we present a minimalistic *Domain Specific Language* for some ARC tasks. We also implement an evaluation function able to run a such program against an input image.

In a second time, we implement a simple genetic algorithm that is able to generate programs written in this DSL and demonstrate its usage on an ARC task.

### Additions

* Added function to increase size of images - provides ability to representation on a 90x90 grid providing space around the image
* Additional scratchpad in the image - hidden colors on top of existing to indicate awareness of patterns to next operators - can be used for identifying filling, size/rank of objects, boundaries, special pixels, other features - this may not be necessary, since operators create multiple images, and other operators combine them.
* create operators 

In [ ]:
import numpy as np
import pandas as pd
import itertools
import random

import os
import json
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import colors

from joblib import Parallel, delayed
import multiprocessing

data_path = Path('/kaggle/input/abstraction-and-reasoning-challenge/')
training_path = data_path / 'training'
test_path = data_path / 'test'

training_tasks = sorted(os.listdir(training_path))

In [ ]:
#
# This code is used to display a task
# It accepts 11 colors, one more than the images, in case we want to use it
#

cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25','#4a4d4a'])
norm = colors.Normalize(vmin=0, vmax=10)
def plot_one(ax, i,train_or_test,input_or_output):
    cmap = colors.ListedColormap(
        ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
         '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25','#4a4d4a'])
    norm = colors.Normalize(vmin=0, vmax=10)
    
    input_matrix = task[train_or_test][i][input_or_output]
    ax.imshow(input_matrix, cmap=cmap, norm=norm)
    ax.grid(True,which='both',color='lightgrey', linewidth=0.5)    
    ax.set_yticks([x-0.5 for x in range(1+len(input_matrix))])
    ax.set_xticks([x-0.5 for x in range(1+len(input_matrix[0]))])     
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_title(train_or_test + ' '+input_or_output)
    

def plot_task(task):
    """
    Plots the first train and test pairs of a specified task,
    using same color scheme as the ARC app
    """    
    num_train = len(task['train'])
    fig, axs = plt.subplots(2, num_train, figsize=(3*num_train,3*2))
    for i in range(num_train):     
        plot_one(axs[0,i],i,'train','input')
        plot_one(axs[1,i],i,'train','output')        
    plt.tight_layout()
    plt.show()        
        
    num_test = len(task['test'])
    fig, axs = plt.subplots(2, num_test, figsize=(3*num_test,3*2))
    if num_test==1: 
        plot_one(axs[0],0,'test','input')
        plot_one(axs[1],0,'test','output')     
    else:
        for i in range(num_test):      
            plot_one(axs[0,i],i,'test','input')
            plot_one(axs[1,i],i,'test','output')  
    plt.tight_layout()
    plt.show() 

    
# Display each output of the function
def show_image_list(images):
    """ Show each image contained in a list. """
    p = plt.figure(figsize=[17,5]).subplots(1, len(images))
    
    if len(images) > 1:
        for i, image in enumerate(images):
            p[i].imshow(image, cmap=cmap, norm=norm)
    elif len(images) == 1:
        p.imshow(images[0], cmap=cmap, norm=norm)


# Domain Specific Language (DSL)**

Our DSL will be a collection of functions of type `np.array -> [np.array]` and `[np.array] -> [np.array]`.

The first kind of function take an image, and produce a list of images (for example, the image split by different colors). The second type of function take a list of images and produce a new list (for exemple, intersect).
[](http://)

## DSL Implementation

We start with the functions that take *one image* and produce an *a list of images*.](http://)

In [ ]:
# np.array -> [np.array]
def groupByColorx_unlifted(pixmap):
    """ Split an image into a collection of images with unique color """
    # Count the number of colors
    nb_colors = int(pixmap.max()) + 1
    # Create a pixmap for each color
    splited = [(pixmap == i) * i for i in range(1, nb_colors)]
    # Filter out empty images
    return [x for x in splited if np.any(x)]

def groupByColor_unlifted(pixmap):
    """ Split an image into a collection of images with unique color """
    # Identify the colors
    rng = np.unique(pixmap)
    nb_colors = int(pixmap.max()) + 1
    # Create a pixmap for each color
    splited = [ (pixmap == i) * i for i in rng]
    #splited.append(pixmap)
    return splited
# 

# np.array -> [np.array]
def cropToContent_unlifted(pixmap):
    """ Crop an image to fit exactly the non 0 pixels """
    # Op argwhere will give us the coordinates of every non-zero point
    true_points = np.argwhere(pixmap)
    if len(true_points) == 0:
        return []
    # Take the smallest points and use them as the top left of our crop
    top_left = true_points.min(axis=0)
    # Take the largest points and use them as the bottom right of our crop
    bottom_right = true_points.max(axis=0)
    # Crop inside the defined rectangle
    res = pixmap[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]
    return [res]

# np.array -> [np.array]
def splitH_unlifted2(pixmap):
    """ Split horizontally an image """
    h = pixmap.shape[0]
    if h % 2 == 1:
        h = h // 2
        return [pixmap[:h,:], pixmap[h+1:,:]]
    else:
        h = h // 2
        return [pixmap[:h,:], pixmap[h:,:]]
    
    
def splitHn_unlifted(pixmap):
    """ Split horizontally an image """
    ## find the split point based on the vertical line in the image
    #look for vertical lines (unchnging color)
    d=np.diff(pixmap, axis=0)
    da=np.sum(np.abs(d), axis=0)  #columns with 0 are split options
    loc=np.where(da == 0)[0]
    im=[]
    for p in loc:
        #print(p)
        im.append(pixmap[:,:p]) # left half no line
        im.append(pixmap[:,p+1:])  # right half with no line

    return(im)

def splitH_unlifted(pixmap):
    """ Split horizontally an image """
    ## find the split point based on the vertical line in the image
    #look for vertical lines (unchnging color)
    d=np.diff(pixmap, axis=0)
    da=np.sum(np.abs(d), axis=0)  #columns with 0 are split options
    loc=np.where(da == 0)[0]
    im=[]
    for p in loc:
        #print(p)
        im.append(pixmap[:,p:]) # right half with line
        im.append(pixmap[:,:p+1]) # left half with line
    return(im)


def splitV_unlifted(pixmap):
    """ Split horizontally an image """
    ## find the split point based on the horizontal line in the image
    #look for horizontal lines (unchnging color)
    d=np.diff(pixmap, axis=1)
    da=np.sum(np.abs(d), axis=1)  #rows with 0 are split options
    loc=np.where(da == 0)[0]
    im=[]
    for p in loc:
        #print(p)
        im.append(pixmap[p:,:]) # bot half with line
        im.append(pixmap[:p+1,:]) # top half with line
    return(im)

def splitVn_unlifted(pixmap):
    """ Split horizontally an image """
    ## find the split point based on the horizontal line in the image
    #look for horizontal lines (unchnging color)
    d=np.diff(pixmap, axis=1)
    da=np.sum(np.abs(d), axis=1)  #rows with 0 are split options
    loc=np.where(da == 0)[0]
    im=[]
    for p in loc:
        #print(p)
        im.append(pixmap[:p,:]) # top half no line
        im.append(pixmap[p+1:,:])  # bot half with no line
    return(im)


# np.array -> [np.array]
def negative_unlifted(pixmap):
    """ Compute the negative of an image (and conserve the color) """
    negative = np.logical_not(pixmap).astype(int)
    color = max(pixmap.max(), 1)
    return [pixmap,negative * color]

def extend_unlifted(pixmap):
    """ Create image where original is padded by 30 pixels all around """
    ##Check if already padded enough?  min dim >=90
    if min(pixmap.shape) < 90 :
        padded=np.pad(pixmap, ((30,30), (30, 30)), 'constant', constant_values=(0))
        #return [padded, pixmap]
        return [padded]
    else:
        #return [pixmap, pixmap]
        return [pixmap]
    


def rotate_unlifted(pixmap):    
    return [pixmap, np.rot90(pixmap)]# rotated image by 90 deg

def mirror_unlifted(pixmap):
    return [pixmap, np.fliplr(pixmap)]# mirror image flip H

def tile2_unlifted(pixmap):
    image =  np.tile(pixmap, (2,2))
    s = image.shape
    h = min(s[0],90)
    v = min(s[1],90)
    #print("h=",h," v=",v)
    return [pixmap, image[:h,:v]]

def tile3_unlifted(pixmap):
    image =  np.tile(pixmap, (3,3))
    s = image.shape
    h = min(s[0],90)
    v = min(s[1],90)
    #print("h=",h," v=",v)
    return [pixmap, image[:h,:v]]

def tile3h_unlifted(pixmap):
    image =  np.tile(pixmap, 3)
    s = image.shape
    h = min(s[0],90)
    v = min(s[1],90)
    #print("h=",h," v=",v)
    return [pixmap, image[:h,:v]]

def tile2h_unlifted(pixmap):
    image =  np.tile(pixmap, 2)
    s = image.shape
    h = min(s[0],90)
    v = min(s[1],90)
    #print("h=",h," v=",v)
    return [pixmap, image[:h,:v]]


def shift_unlifted(pixmap):
    return # shift image over by 1 to the right

def zoom3_unlifted(pixmap):
    
    newshape = np.array(pixmap.shape) * 3
    slices = [ slice(0,old, float(old)/new) for old,new in zip(pixmap.shape,newshape) ]
    coordinates = np.mgrid[slices]
    indices = coordinates.astype('i')   #choose the biggest smaller integer index
    
    return [pixmap, pixmap[tuple(indices)]] # enlarge image taking each 1 pixel and making it 3x3


def zoom2_unlifted(pixmap):
    
    newshape = np.array(pixmap.shape) * 2
    slices = [ slice(0,old, float(old)/new) for old,new in zip(pixmap.shape,newshape) ]
    coordinates = np.mgrid[slices]
    indices = coordinates.astype('i')   #choose the biggest smaller integer index
    
    return [pixmap, pixmap[tuple(indices)]] # enlarge image taking each 1 pixel and making it 2x2




def lzoom_unlifted(pixmap):

    return #take small image and turn 1 pixel  2 horizontal, 1 into 3 horizontal  (with mirror and flip this can expand to other cases)



In [ ]:
# [np.array] -> [np.array]
def identity(x: [np.array]):
    return x

# [np.array] -> [np.array]
def tail(x):
    if len(x) > 1:
        return x[1:]
    else:
        return x

def head(x):
    if len(x) > 1:
        return x[:-1]
    else:
        return x    
    
def swap(x):
    if len(x) > 1:
        t=x[-1]
        x[-1]=x[-2]
        x[-2]=t
    
    return x  


# [np.array] -> [np.array]
def init(x):
    if len(x) > 1:
        return x[:1]
    else:
        return x

# [np.array] -> [np.array]
def union2(x):
    """ Compute the pixel union of all images in the list. """
    if len(x) < 2:
        return x
    
    # Make sure everybody have the same shape
    first_shape = tuple(x[0].shape)
    for pixmap in x[1:]:
        if first_shape != tuple(pixmap.shape):
            return []
    
    return [np.bitwise_or.reduce(np.array(x).astype(int))]
    
def intersect2(x):
    """ Compute the pixel intersection of all images in the list. """
    if len(x) < 2:
        return x
    
    # Make sure everybody have the same shape
    first_shape = tuple(x[0].shape)
    for pixmap in x[1:]:
        if first_shape != tuple(pixmap.shape):
            return []
    
    return [(np.prod(np.array(x), axis=0) > 0).astype(int)]

def union(x):
    
    if len(x) < 2:
        return x
    l={}
    # search list to identify shapes and counts
    for pixmap in x:
        s=str(pixmap.shape)
        try:
            l[s].append(pixmap)
        except KeyError:
            l[s]=[pixmap]
    
    im=[]
    for i, k in enumerate(l):
    #print(i, k,len(l[k]))
        if len(l[k])>1:
            mask = (np.logical_or.reduce(np.array(l[k]))).astype(int)
            #print(mask)
            for j,m in enumerate(l[k]):
                #print()
                im.append(np.prod([mask, m], axis=0) )

    return im
    
    
    
    
def intersect(x):
    
    if len(x) < 2:
        return x
    l={}
    # search list to identify shapes and counts
    for pixmap in x:
        s=str(pixmap.shape)
        try:
            l[s].append(pixmap)
        except KeyError:
            l[s]=[pixmap]
    
    im=[]
    for i, k in enumerate(l):
    #print(i, k,len(l[k]))
        if len(l[k])>1:
            mask = (np.prod(np.array(l[k]), axis=0) > 0).astype(int)
            for j,m in enumerate(l[k]):
                #print()
                im.append(np.prod([mask, m], axis=0) )

    return im


def sortByColor(xs):
    """ Sort pictures by increasing color id. """
    xs = [x for x in xs if len(x.reshape(-1)) > 0]
    return list(sorted(xs, key=lambda x: x.max()))

def sortByWeight(xs):
    """ Sort images by how many non zero pixels are contained. """
    xs = [x for x in xs if len(x.reshape(-1)) > 0]
    return list(sorted(xs, key=lambda x: (x>0).sum()))

def reverse(x):
    """ Reverse the order of a list of images. """
    return x[::-1]

def colorshift(x):
    
    im=[]
    for pixmap in x:
        pixmap=pixmap+1
        pixmap[pixmap==1] = 0 # turn black to black again
        pixmap[pixmap>10] = 1 # rotate to 1
        im.append(pixmap)
    return im

def stackv(x):  ## join images top to bottom
    if len(x) < 2:
        return x
    l={}
    # search list to identify shapes and counts
    for pixmap in x:
        s=pixmap.shape[1]
        try:
            l[s].append(pixmap)
        except KeyError:
            l[s]=[pixmap]
    
    im=[]
    for i, k in enumerate(l):
    #print(i, k,len(l[k]))
        if len(l[k])>1: ## more than one image with this dimension
            im.append(np.vstack(l[k]) )

    return im

def stackh(x):  ## join images top to bottom
    if len(x) < 2:
        return x
    l={}
    # search list to identify shapes and counts
    for pixmap in x:
        s=pixmap.shape[0]
        try:
            l[s].append(pixmap)
        except KeyError:
            l[s]=[pixmap]
    
    im=[]
    for i, k in enumerate(l):
    #print(i, k,len(l[k]))
        if len(l[k])>1: ## more than one image with this dimension
            im.append(np.hstack(l[k]) )

    return im


## Composition of functions

It is important to make sure we can chain both functions. It is clear how we can compose two functions `f` and `g` of type `[np.array] -> [np.array]` ; We symply call `g(f([input_image]))`.


For each function of the first type, we need to generated a *lifted version*. A function `np.array -> [np.array]` is can be turned into a function of type `[np.array] -> [np.array]` simply by applying the first function on each image and concatenating the results.

---
If you want to know more about the `lift` function, have a look to the concept of [*monades*](https://en.wikipedia.org/wiki/Monad_%28functional_programming%29). We are indeed using the *list monade*.

In [ ]:
def lift(fct):
    # Lift the function
    def lifted_function(xs):
        list_of_results = [fct(x) for x in xs]
        return list(itertools.chain(*list_of_results))
    # Give a nice name to the lifted function
    import re
    lifted_function.__name__ = re.sub('_unlifted$', '_lifted', fct.__name__)
    return lifted_function

cropToContent = lift(cropToContent_unlifted)
groupByColor = lift(groupByColor_unlifted)
splitH = lift(splitH_unlifted)
splitHn = lift(splitHn_unlifted)
splitV = lift(splitV_unlifted)
splitVn = lift(splitVn_unlifted)
negative = lift(negative_unlifted)
extend = lift(extend_unlifted)
rotate = lift(rotate_unlifted)
mirror = lift(mirror_unlifted)
tile2 = lift(tile2_unlifted)
tile3 = lift(tile3_unlifted)
tile2h = lift(tile2h_unlifted)
tile3h = lift(tile3h_unlifted)
zoom3 = lift(zoom3_unlifted)
zoom2 = lift(zoom2_unlifted)

# Task

We now load a simple task and execute one of our functions on it.

# Program evaluation


We define our building blocks for programs (the functions in our DSL). We will define a program as a list of functions from our DSL ; `program: [[np.array] -> [np.array]]`. The instructions in our programs will be executed *from left to right*. This mean that if we want to first `splitByColor` and then compute the `negative` of the image, we need to write `[splitByColor, negative]` in this order.

Let's first write an utilitary function to describe a program as a human readable string.

In [ ]:
def program_desc(program):
    """ Create a human readable description of a program. """
    desc = [x.__name__ for x in program]
    return(' >> '.join(desc))

# Display the program description alongside its output
#program = [splitH, groupByColor, negative, intersect]
#print(program_desc(program))

## The evaluation method
We need a way to run a such program on a pictures and recover the result. This is done by the `evaluate` function.

In [ ]:
def evaluate(program: [], input_image: np.array):
    # Make sure the input is a np.array
    input_image = np.array(input_image)
    assert type(input_image) == np.ndarray
    
    # Apply each function on the image
    image_list = [input_image]
    for fct in program:
        # Apply the function
        #image_list.append(input_image)  ##try this to get input in every step
        image_list = fct(image_list)
        # Filter out empty images
        image_list = [img for img in image_list if img.shape[0] > 0 and img.shape[1] > 0]
        # Break if there is no data
        if image_list == []:
            return []
    return image_list        

# Program generation

We now have a simple and powerful language to express various transformation on images. But someone or something still have to write the actual program that can solve a task. In this part, we will implement a naive but somewhat efficient genetic algorithm that will be able to find by itself the solution to a task.

## Is a program solution

First, we need a way to know if a program is a solution of the given examples of a task.

In [ ]:
# Load my favorite task
t=0
task_file = str(training_path / training_tasks[t])
with open(task_file, 'r') as f:
    task = json.load(f)
    plot_task(task)

In [ ]:
def are_two_images_equals(a, b):
    if tuple(a.shape) == tuple(b.shape):
        if (np.abs(b-a) < 1).all():
            return True
    return False

def is_solution(program, task, verbose=True):
    for sample in task: # For each pair input/output
        
        i = np.array(sample['input'])
        o = np.array(sample['output'])

        # Evaluate the program on the input
        images = evaluate(program, i)
        if len(images) < 1:
            return False
        
        # The solution should be in the 3 last outputs
        images = images[-3:]
        #print("Images=",images) #debug
        # Check if the output is in the 3 images produced
        is_program_of_for_sample = any([are_two_images_equals(x, o) for x in images])
        if not is_program_of_for_sample:
            return False
    
    return True

program = [zoom3,tile3,intersect]
print(program_desc(program),"is a solution of the task:", is_solution(program, task['train']))

In [ ]:
results = evaluate(program=[zoom3,tile3,intersect], input_image=task['train'][0]['input'])
show_image_list(results)

In [ ]:
show_image_list(results[-3:])

## Fitness

To help our algorithm progress in the right direction, we need a way to give a score to an existing program. The smaller is the score of the program, the closer we are to the solution. One can think of this score as a distance of our program to the optimal solution.

Notice that one can think of this program as a minimization problem (minimize `score`) or maximization problem (minimize `-score`). On machine learning it is common to minimise a distance wereas in genetic algorithm literature you can read that we maximize the fitness of an agent. Both convention work perfectly, but it is more convenient if we choose one and stick to it. Therefore, we will MINIMIZE the score of our programs.

First, we are going to evaluate how our program perform on different aspects.

In [ ]:
import re 
def subimg_location(haystack, needle):
    
    try:
        
        haystack_str = b"".join(48+haystack.flatten().astype(np.dtype('b'))).decode("latin-1") 
        needle_str = b"".join((48+needle.flatten()).astype(np.dtype('b'))).decode("latin-1") 
    except:
        print("needle=",needle)
        print("type=",type(needle))
        print("shape=",needle.shape)
        print("temp=",temp)
        print(" ")
    

    gap_size = (haystack.shape[1] - needle.shape[1]) 
    
    #print(gap_size)
    gap_regex = '.{' + str(gap_size) + '}'

    #print(gap_regex)
    # Split b into needle.size[0] chunks
    chunk_size = needle.shape[1] 
    split = [needle_str[i:i+chunk_size] for i in range(0, len(needle_str), chunk_size)]

    # Build regex
    regex = re.escape(split[0])
    for i in range(1, len(split)):
        regex += gap_regex + re.escape(split[i])

    p = re.compile(regex)
    m = p.search(haystack_str)

    if not m:
        return False

    x, _ = m.span()

    left = x % (haystack.shape[1] ) 
    top  = int(x / haystack.shape[1] )

    return (top, left)

In [ ]:
def width_fitness(predicted, expected_output):
    """ How close the predicted image is to have the right width. Less is better."""
    return np.abs(predicted.shape[0] - expected_output.shape[0])

def height_fitness(predicted, expected_output):
    """ How close the predicted image is to have the right height. Less is better."""
    return np.abs(predicted.shape[1] - expected_output.shape[1])

def activated_pixels_fitness(p, e):
    """ How close the predicted image to have the right pixels. Less is better."""
    shape = (max(p.shape[0], e.shape[0]), max(p.shape[1], e.shape[1]))
    diff = np.zeros(shape, dtype=int)
    diff[0:p.shape[0], 0:p.shape[1]] = (p > 0).astype(int)
    diff[0:e.shape[0], 0:e.shape[1]] -= (e > 0).astype(int)
    
    fit = (diff != 0).sum()
    
    return fit


def submatch_fitness(p,e):
    
    (py,px) = p.shape
    (ey,ex) = e.shape
    
    if px<=ex and py<=ey:
        
        r = subimg_location(haystack=e,needle=p)
    
    elif px>=ex and py>=ey:
        r = subimg_location(haystack=p,needle=e)
    elif px<ex and py>=ey:
        
        r = subimg_location(haystack=e[:,:px],needle=p)
    elif px>=ex and py<ey:
 
        r = subimg_location(haystack=e[:py,:],needle=p)
    else:
        print("whats up",px,ex,py,ey)
            
    if r == False:
        return 1
    else:
        #print(r)
        return 0
        

def pixels_match_fitness(p,e):
    shape = (max(p.shape[0], e.shape[0]), max(p.shape[1], e.shape[1]))
    #check how many pixels match the expected output.
    p1 = np.zeros(shape, dtype=int)
    e1 = np.zeros(shape, dtype=int)
    
    p1[0:p.shape[0], 0:p.shape[1]] = p
    e1[0:e.shape[0], 0:e.shape[1]] = e
    
    #q=np.abs(p1-e1).sum() + abs(e.shape[0]-p.shape[0])*abs(e.shape[1]-p.shape[1])
    q=np.abs(p1-e1).sum()
    
    s=submatch_fitness(p,e)
    
    return q*(1+5*s)
    

def colors_fitness(p, e):
    p_colors = np.unique(p)
    e_colors = np.unique(e)
    
    nb_inter = len(np.intersect1d(p_colors, e_colors))

    return (len(p_colors) - nb_inter) + (len(e_colors) - nb_inter)

fitness_functions = [colors_fitness, activated_pixels_fitness, height_fitness, width_fitness, pixels_match_fitness]


The fitness score (less is better) of our function will be a 4-dimensional tuple containing the result of each of the fitness functions.

We want to be able to compare two score. Unfortunately, the *lixocographical order* is not adapted, as there is no reason than having a small `width score` is better than having a small `height score`. We are going to define a partial order that give the same weight to any fitness function.

When we compare two tuple with this partial order, `(3, 2, 4, 0) < (3, 2, 5, 0)` and `(3, 2, 4, 0) < (4, 2, 4, 0)`. But there is no way to compare `(3, 2, 5, 0)` and `(4, 2, 4, 0)`. We say this two values are *incomparable*. If two score are incomparable, it means that we cannot say that one program is better than the over.

In [ ]:
def product_less(a, b):
    """ Return True iff the two tuples a and b respect a<b for the partial order. """
    a = np.array(a)
    b = np.array(b)
    return (np.array(a) < np.array(b)).all()
    

We now write a function that evaluate the fitness of a program on a task.

In [ ]:
# ([[np.array] -> [np.array]], Taks) -> (int, int, ..., int)
def evaluate_fitness(program, task):
    """ Take a program and a task, and return its fitness score as a tuple. """
    score = np.zeros((len(fitness_functions)))
    
    # For each sample
    for sample in task:
        i = np.array(sample['input'])
        o = np.array(sample['output'])
        images = evaluate(program, i)
        # For each fitness function
        for index, fitness_function in enumerate(fitness_functions):
            
            if images == []: # Penalize no prediction!
                score[index] += 500
            else: # Take only the score of the last 3 outputs
                score[index]=0
                im3=images[-3:]
                for imct in range(len(im3)):
                    score[index] = score[index] + fitness_function(im3[imct], o)/len(images)
                
    return tuple(score)

print("Fitness evaluation:", evaluate_fitness([splitHn,reverse,intersect,colorshift], task['train']))

## Asexual reproduction

Now that we can compare two programs we need a way to generate some of them. We will generate them randomly from a pool of best candidate.

For the initial run, and also to be able to evaluate fresh candidates, we will also allow spontaneous generation of new born one instruction programs.

In [ ]:
def build_candidates(allowed_nodes=[identity], best_candidates=[], nb_candidates=50):
    """
    Create a poll of fresh candidates using the `allowed_nodes`.
    
    The pool contain a mix of new single instructions programs
    and mutations of the best candidates.
    """
    new_candidates = []
    length_limit = 5 # Maximal length of a program
    
    def random_node():
        return random.choice(allowed_nodes)
    
    # Until we have enougth new candidates
    while(len(new_candidates) < nb_candidates):
        # Add 10 new programs
        for i in range(5):
            new_candidates += [[random_node()]]
        
        # Create new programs based on each best candidate
        for best_program in best_candidates:
            # Add one op on its right but limit the length of the program
            if len(best_program) < length_limit - 1:
                new_candidates += [[random_node()] + best_program]
            # Add one op on its left but limit the length of the program
            if len(best_program) < length_limit - 1:
                new_candidates += [best_program + [random_node()]]
            # Mutate one instruction of the existing program
            new_candidates += [list(best_program)]
            new_candidates[-1][random.randrange(0, len(best_program))] = random_node()
   
    # Truncate if we have too many candidates
    np.random.shuffle(new_candidates)
    return new_candidates[:nb_candidates]

# Test the function by building some candidates
#len(build_candidates(allowed_nodes=[identity], best_candidates=[[identity]], nb_candidates=42))

## Find a program given a task

This is the last step to our genetic algorithm. We have all the building blocks:
 * Generating both new programs and mutation of existing solutions
 * Evaluating the fitness score of a program
 * Comparing two programs to know if one perform better than the other
 * Detecting when a solution was found
 
We can now write a function that will keep generating programs with increasing complexity until a solution is found.

Using our partial order, we are going to keep the best candidates. Because the order is partial,
there is no bound on how many uncomparables candidates we may have at a given iteration.

In [ ]:
def build_model(task, max_iterations=20, verbose=True):
    candidates_nodes = [
        intersect,
        
        cropToContent, groupByColor, splitH, splitV,splitVn, splitHn,
        tile2,tile3,zoom3,zoom2,union,sortByColor, mirror,rotate,colorshift,
        tile2h,tile3h,stackh,stackv,head,
        init,tail, sortByWeight, reverse, negative,swap
        
        #extend,push,pull

    ]
    
    print(".")
    if verbose:
        print("Candidates nodes are:", [program_desc([n]) for n in candidates_nodes])
        print()

    best_candidates = {} # A dictionary of {score:candidate}
    for i in range(max_iterations):
        if verbose:
            print("Iteration ", i+1)
            print("-" * 10)
        
        # Create a list of candidates
        candidates = build_candidates(candidates_nodes, best_candidates.values())
        
        # Keep candidates with best fitness.
        # They will be stored in the `best_candidates` dictionary
        # where the key of each program is its fitness score.
        for candidate in candidates:
            score = evaluate_fitness(candidate, task)
            is_uncomparable = True # True if we cannot compare the two candidate's scores
            
            # Compare the new candidate to the existing best candidates
            best_candidates_items = list(best_candidates.items())
            for best_score, best_candidate in best_candidates_items:
                if product_less(score, best_score):
                    # Remove previous best candidate and add the new one
                    del best_candidates[best_score]
                    best_candidates[score] = candidate
                    is_uncomparable = False # The candidates are comparable
                if product_less(best_score, score) or best_score == score:
                    is_uncomparable = False # The candidates are comparable
            if is_uncomparable: # The two candidates are uncomparable
                best_candidates[score] = candidate

        # For each best candidate, we look if we have an answer
        for program in best_candidates.values():
            if is_solution(program, task):
                return program
            
        # Give some informations by selecting a random candidate
        if verbose:
            print("Best candidates length:", len(best_candidates))
            random_candidate_score = random.choice(list(best_candidates.keys()))
            print("Random candidate score:", random_candidate_score)
            print("Random candidate implementation:", program_desc(best_candidates[random_candidate_score]))
    return None

In [ ]:
# Load my favorite task
task_file = str(training_path / training_tasks[5])
print(task_file)
with open(task_file, 'r') as f:
    task = json.load(f)
    plot_task(task)

In [ ]:
program = build_model(task['train'],max_iterations=100, verbose=True)

print()
if program is None:
    print("No program was found")
else:
    print("Found program:", program_desc(program))

In [ ]:
submission = pd.read_csv(data_path / 'sample_submission.csv', index_col='output_id')
display(submission.head())

In [ ]:
def flattener(pred):
    str_pred = str([row for row in pred])
    str_pred = str_pred.replace(', ', '')
    str_pred = str_pred.replace('[[', '|')
    str_pred = str_pred.replace('][', '|')
    str_pred = str_pred.replace(']]', '|')
    return str_pred


In [ ]:
example_grid = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
display(example_grid)
print(flattener(example_grid))

* To Do - try each with 20 iterations and then work on harder ones in a second pass
* Add time monitoring for 9 hrs
* return best 3 programs when nothing found

In [ ]:
#read all inputs into memory first
taskdict={}


for output_id in submission.index:
    task_id = output_id.split('_')[0]
    pair_id = int(output_id.split('_')[1])
    f = str(test_path / str(task_id + '.json'))
    with open(f, 'r') as read_file:
        task = json.load(read_file)
        
    taskdict[output_id] = task


In [ ]:

num_cores = multiprocessing.cpu_count()
print(num_cores)

results = Parallel(n_jobs=num_cores)(delayed(build_model)(taskdict[i]['train'],50,False) for i in submission.index)


In [ ]:
ct=0
resdict={}
for output_id in submission.index:
    resdict[output_id] = results[ct]
    ct=ct+1

In [ ]:
for output_id in submission.index:
    task_id = output_id.split('_')[0]
    pair_id = int(output_id.split('_')[1])
    f = str(test_path / str(task_id + '.json'))
    
    task = taskdict[output_id]
        
    program = resdict[output_id]
    if program is None:
        print("No program was found for:",f)
        
        # skipping over the training examples, since this will be naive predictions
        # we will use the test input grid as the base, and make some modifications
        data = task['test'][pair_id]['input'] # test pair input
        # for the first guess, predict that output is unchanged
        pred_1 = flattener(data)
        # for the second guess, change all 0s to 5s
        data = [[5 if i==0 else i for i in j] for j in data]
        pred_2 = flattener(data)
        # for the last gues, change everything to 0
        data = [[0 for i in j] for j in data]
        pred_3 = flattener(data)
        pred = pred_1 + ' ' + pred_2 + ' ' + pred_3 + ' ' 
        
    else:
        count=count+1
        print("Task:",f," total:", count," Found program:", program_desc(program))
        results = evaluate(program=program, input_image=task['test'][pair_id]['input'])
        pred=""
        ct=min(3,len(results))
        for i in range(ct):
            x=results[i]
            x[x>9] = 0 # change extra color to background if it exists
            pred = pred + flattener(x.to_list()) + ' '
         
    submission.loc[output_id, 'output'] = pred

submission.to_csv('submission.csv')